### Imports

In [ ]:
import scipy as sci
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
%matplotlib widget

### Read Data

In [ ]:
file = 'Iris.csv'
data = pd.read_csv(file, index_col=0, header=0).reset_index(drop=True)
N_S = len(data.index)
indices = data.index

### Preprocess

In [ ]:
X_S, Y_S = data.iloc[:, :-1], data.iloc[:, -1]

def normalize(points):
    min = points.min(axis=0)
    scale = points.max(axis=0) - min
    def denormalize(points):
        return points * scale + min
    return (points - min) / scale, denormalize

X_S, denormalize = normalize(X_S)

def cat_to_int(data):
    cat = data.unique()
    cat_convert = dict(zip(cat, range(len(cat))))
    cat_revert = dict(zip(range(len(cat)), cat))
    return data.apply(lambda x: cat_convert[x]), cat_revert

Y_S, cat_revert = cat_to_int(Y_S)


### Learner measures

In [ ]:
# lower bound to sample size
def PAC_eta(H_norm, delta, epsilon):
    return (np.ln(H_norm) - np.ln(delta))/epsilon

# lower bound to generalization error


def PAC_delta(H_norm, epsilon, eta):
    return H_norm/np.exp(eta*epsilon)


# Sampling

## Distributions

In [ ]:
pdf_names = ["Uniform", "Triangular", "Normal"]


def pdf(pdf, x): return {
    "Uniform": sci.stats.uniform.pdf(x, 0, N),
    "Triangular": sci.stats.triang.pdf(x, 0.5, -1, N+1),
    "Normal": sci.stats.norm.pdf(x, N//2, N//6)}[pdf]


plt.figure()
for name in pdf_names:
    plt.plot(x, pdf(name, x), label=name)
plt.legend()
plt.title("Probability Density Function")
plt.xlabel("Sample Index")

plt.figure()
for name in pdf_names:
    p = pdf(name, x)
    I = np.log2(1/p)
    H = np.sum(-p*np.log2(p))
    plt.plot(x, I, label=f"{name}: {H:.2f}")
plt.legend(title="$\\bf{Entropy}$")
plt.title("Information content")
plt.xlabel("Sample Index")


## Comparison of samples

In [ ]:
def set_axis_labels(ax, x, y, z):
    ax.set_xlabel(x)
    ax.set_ylabel(y)
    ax.set_zlabel(z)


for name in pdf_names:
    plt.figure()
    ax = plt.axes(projection='3d', title=f"{name} Sampling")
    sample = data.sample(frac=0.6, weights=pdf(name, x))
    total = 0
    for (k, v) in cats1.items():
        total += np.sum(sample["Species"] == v)
        p = sample[sample["Species"] == v]
        ax.scatter(*zip(*p.iloc[:, 0:3].values.tolist()),
                   label=f"{k}: {len(p)}")
    plt.legend(title=f"Samples: {total}")
    set_axis_labels(ax, *p.columns[0:3])


# Training

In [ ]:
# gets B such that ||Y-XB||^2 is minimized
def regression(X, Y):
    # Adds a 1s column to X to represent a constant parameter
    X = np.append(np.ones(X.shape[0])[:, None], X, axis=1)
    Beta = np.asmatrix(X.T @ X).I @ X.T @ Y
    return Beta


sample = data.sample(frac=0.6, weights=pdf("Uniform", x))
X = sample.iloc[:, :-1]
Y = sample.iloc[:, -1]

X = normalize(X)
Beta = regression(X, Y)
a = np.mgrid[0:1:0.1, 0:1:0.1, 0:1:0.1, 0:1:0.1]
X = np.append(np.ones(X.shape[0])[:, None], X, axis=1)
Z = np.round(X@Beta)
A = pd.DataFrame(X[:, 1:], index=Y.index)
A['Z'] = Z

plt.figure()
ax = plt.axes(projection='3d', title="Linear Regression")
total = 0
errors = 0
for (k, v) in cats1.items():
    TP = (A["Z"] == v) & (Y == v)
    FP = (A["Z"] == v) & (Y != v)
    errors += np.sum(FP)
    total += np.sum(TP)
    p = A[TP]
    c = ax.scatter(*zip(*p.iloc[:, 0:3].values.tolist()), marker="o",
                   label=f"{k}: {len(p)}/{np.sum(FP)}").get_facecolors()[0]
    if np.sum(FP) > 0:
        p = A[FP]
        ax.scatter(*zip(*p.iloc[:, 0:3].values.tolist()), marker="x", color=c)
plt.legend(title=f"Matches/Errors: {total}/{errors}")
set_axis_labels(ax, *p.columns[0:3])
